In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
# 1. Generate sample data
np.random.seed(42)

num_samples = 1000

# Inputs: height (150-200 cm), weight (40-100 kg), income (20k-100k)
X = np.random.rand(num_samples, 3)
X[:, 0] = X[:, 0] * 50 + 150     # height
X[:, 1] = X[:, 1] * 60 + 40      # weight
X[:, 2] = X[:, 2] * 80000 + 20000  # income

In [ ]:
# Output 1: Age (regression, 18–60 years)
y_age = X[:, 0] * 0.3 + X[:, 1] * 0.2 + X[:, 2] * 0.0001 + np.random.randn(num_samples) * 2
y_age = np.clip(y_age, 18, 60)

# Output 2: City (classification - Delhi, Mumbai, Bangalore)
cities = np.random.choice(['Delhi', 'Mumbai', 'Bangalore'], size=num_samples)
label_encoder = LabelEncoder()
y_city = label_encoder.fit_transform(cities)
y_city_cat = to_categorical(y_city, num_classes=3)

In [ ]:
# 2. Split into train/test
X_train, X_test, y_age_train, y_age_test, y_city_train, y_city_test = train_test_split(
    X, y_age, y_city_cat, test_size=0.2, random_state=42
)


In [ ]:
# 3. Functional API model
inputs = Input(shape=(3,))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)

In [ ]:
# Output 1: Age (regression)
age_output = Dense(1, name='age_output')(x)

In [ ]:
# Output 2: City (classification)
city_output = Dense(3, activation='softmax', name='city_output')(x)

In [ ]:
# Build model
model = Model(inputs=inputs, outputs=[age_output, city_output])

In [ ]:
# 4. Compile the model
model.compile(
    optimizer='adam',
    loss={
        'age_output': 'mse',
        'city_output': 'categorical_crossentropy'
    },
    metrics={
        'age_output': 'mae',
        'city_output': 'accuracy'
    }
)

In [ ]:
# 5. Train the model
history = model.fit(
    X_train, {'age_output': y_age_train, 'city_output': y_city_train},
    validation_data=(X_test, {'age_output': y_age_test, 'city_output': y_city_test}),
    epochs=20,
    batch_size=32
)

In [ ]:
# 6. Evaluate the model
loss, age_loss, city_loss, age_mae, city_acc = model.evaluate(
    X_test, {'age_output': y_age_test, 'city_output': y_city_test}
)
